In [57]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

from playground.load import dataset, clip_model, clip_processor, model_device, get_image_features

In [2]:
from tqdm.auto import tqdm

In [56]:
# df.to_parquet('data/image_meta.parquet')
# np.save('data/image_vectors.npy', image_vectors)
df = pd.read_parquet('data/image_meta.parquet')
image_vectors = np.load('data/image_vectors.npy')

In [58]:
def get_y_true(df, label):
    assert label in df.label.unique()
    y_true = np.zeros(df.shape[0])
    y_true[df[df.label == label].index.values] =1
    return y_true

def get_query_vector(text):
    query_tokens = clip_processor(text=[text], return_tensors='pt')
    query_vector = clip_model.get_text_features(query_tokens['input_ids'].to(model_device))
    query_vector = F.normalize(query_vector)
    query_vector = query_vector.cpu().detach().numpy().reshape(-1)
    return query_vector

In [59]:
from sklearn.metrics import average_precision_score

In [65]:
y_true = get_y_true(df, 'banana')
query_vector = get_query_vector('a photo of a banana')
image_scores = image_vectors @ query_vector
average_precision_score(y_true, image_scores)